# XAUUSD

In [145]:
import quandl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
import os

import warnings
import logging

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
xau_ratios = [
    "WGC/GOLD_DAILY_USD"
#     "WGC/GOLD_DAILY_EUR",
#     "WGC/GOLD_DAILY_TRY",
#     "WGC/GOLD_DAILY_JPY",
#     "WGC/GOLD_DAILY_GBP",
#     "WGC/GOLD_DAILY_CAD",
#     "WGC/GOLD_DAILY_CHF",
#     "WGC/GOLD_DAILY_VND",
#     "WGC/GOLD_DAILY_KRW",
#     "WGC/GOLD_DAILY_RUB",
#     "WGC/GOLD_DAILY_AUD",
]

economic_indc = []

### Quandl data terms

Anyone seeking to use this code must first apply for an account with [Quandl](https://www.quandl.com) in order to receive an valid authetitciation key.

In [3]:
DIR_NAME = os.path.abspath(os.path.join(os.getcwd(), '..'))
FILEPATH = os.path.join(DIR_NAME, "auth.txt")

with open(FILEPATH, "r") as f:
    authtoken = f.read();

# Retrieve Data

In [52]:
xau_df_dict = {}


for ratio in tqdm(xau_ratios):
    name = ratio.lower().replace("/", "_")

    # get the ratio dataframe
    df = quandl.get(ratio, authtoken=authtoken, start_date = "1979-01-01")
    df.columns = ["price"]
    
    # check for missing business days 
    if pd.infer_freq(df.index) != "B":
        logging.warn("Datetime frequency is not Business Days")
    
    xau_df_dict[name] = df

100%|██████████| 1/1 [00:02<00:00,  2.90s/it]


## Volatility

In [25]:
annualization_factor = 252.
window_size = [5, 20, 60, 120, 252]

for ratio, df in tqdm(xau_df_dict.items()):
    start_date, end_date = df.index[0], df.index[-1]
    full_range = pd.date_range(start_date, end_date, freq = "B")
    
    if not np.array_equal(df.index, pd.date_range(start_date, end_date, freq="B")):
        logging.warning("\n{} is missing business days".format(ratio))

    for window in window_size:
        df['{}d_market_vol'.format(window)] = np.sqrt(
            (annualization_factor/window) * df['price'].rolling(window).var(ddof=0))

100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


## Quandl Features

In [26]:
features = [
    "FRED/T10Y2Y",
    "ML/AAAEY",
    "ML/CCCY",
    "RATEINF/INFLATION_USA",
]

In [108]:
for ratio, df in xau_df_dict.items():
    for feature in features:
        col_name = feature.lower().replace('/', '_')
        
        # get quandl features. `end_date` is set to df.index[-1] to match the price data
        data = quandl.get(feature, authtoken=authtoken, start_date = "1979-01-01", end_date = df.index[-1])
        start_date, end_date = data.index[0], data.index[-1]

        # Some features contain missing data. To best simulate how the data would be ingested
        # realtime, the current value is forward filled. This achieved by resampling.
        if not np.array_equal(data.index, pd.date_range(start_date, end_date, freq="B")):
            logging.warning("\n\t{} is missing business days".format(feature))
                    
        df[col_name] = data
    
    df[:] = df.ffill()

	FRED/T10Y2Y is missing business days
	ML/AAAEY is missing business days
	ML/CCCY is missing business days
	RATEINF/INFLATION_USA is missing business days


## Technical indicator features

In [114]:
import talib

technical_indicators= {
    "MACD" : ("macd", "macdsignal", "macdhist"),
    "STOCHRSI" : ("fastk", "fastd"),
    "MOM" : ("real")
}

for ratio, df in xau_df_dict.items():
    
    # talib requires market price data. starting price of $1 is taken
    # since absolute values are not important (preprocess scaling)
    price = df['price'].values

    for indicator, indicator_type in technical_indicators.items():
        # Return the result for each indicator
        result = getattr(talib, indicator)(price)

        if isinstance(result, np.ndarray):
            df[indicator.lower()] = result
        else:
            for f, r in zip(indicator_type, result):
                df["{}_{}".format(indicator.lower(), f)] = r

### Columns which specific `fillna` criteria

In [123]:
for ratio, df in xau_df_dict.items():
    fill_na_cols = ['ml_aaaey', 'ml_cccy']
    
    # fillna using a dictionary of average values
    df[fill_na_cols] = df[fill_na_cols].fillna(df[fill_na_cols].mean())

## Data preporocessing 

In [157]:
def train_valid_test_split(all_x, all_y, train_size, valid_size, test_size):
    """
    Generate the train, validation, and test dataset.

    Parameters
    ----------
    all_x : DataFrame
        All the input samples
    all_y : Pandas Series
        All the target values
    train_size : float
        The proportion of the data used for the training dataset
    valid_size : float
        The proportion of the data used for the validation dataset
    test_size : float
        The proportion of the data used for the test dataset

    Returns
    -------
    x_train : DataFrame
        The train input samples
    x_valid : DataFrame
        The validation input samples
    x_test : DataFrame
        The test input samples
    y_train : Pandas Series
        The train target values
    y_valid : Pandas Series
        The validation target values
    y_test : Pandas Series
        The test target values
    """
    
    assert train_size > 0 and train_size < 1.0
    assert valid_size > 0 and valid_size < 1.0
    assert test_size > 0 and test_size < 1.0
    assert train_size + valid_size + test_size == 1.0
    
    dates = all_x.index
    
    train_dates, valid_dates, test_dates = np.split(
        dates, [int(len(dates) * train_size), 
                int(len(dates) * (train_size + test_size))]
    )
    
    X_train = all_x.loc[train_dates, :]
    X_valid = all_x.loc[valid_dates, :]
    X_test = all_x.loc[test_dates, :]
    
    y_train = all_y.loc[train_dates]
    y_valid = all_y.loc[valid_dates]
    y_test = all_y.loc[test_dates]
    
    return {
        "X_train" : X_train, 
        "X_valid" : X_valid, 
        "X_test" : X_test, 
        "y_train" : y_train, 
        "y_valid" : y_valid, 
        "y_test" : y_test
    }

In [164]:
data = xau_df_dict['wgc_gold_daily_usd']
data['target'] = df['price'].shift(-1).pct_change()
data.dropna(inplace = True)

X = data.drop(['price', 'target'], axis = 1)
y = data['target']
               
split_data = train_valid_test_split(X, y, 0.7, 0.15, 0.15)

In [167]:
scalar = StandardScaler()
scalar.fit(split_data["X_train"])

scaled_split_data = {name : scalar.transform(df) if "X" in name else df for name, df in split_data.items()}

In [169]:
n_days = scaled_split_data['X_train'].shape[0]
n_features = scaled_split_data['X_train'].shape[1]

clf_parameters = {
    'criterion': 'entropy',
    'min_samples_leaf': 10,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': 0}

n_trees_l = [30, 100, 250, 500, 1000]

In [ ]:
train_score = []
valid_score = []
oob_score = []
feature_importances = []

for n_trees in tqdm(n_trees_l, desc='Training Models', unit='Model'):
    
    clf = RandomForestClassifier(n_trees, **clf_parameters)
    clf.fit(scaled_split_data['X_train'], scaled_split_data["y_train"])
    
    train_score.append(clf.score(scaled_split_data['X_train'], scaled_split_data["y_train"].values))
    valid_score.append(clf.score(scaled_split_data["X_valid"], scaled_split_data["y_valid"].values))
    
    oob_score.append(clf.oob_score_)
    feature_importances.append(clf.feature_importances_)